# Import Library 


In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 

In [2]:
tf.__version__

'2.12.0'

# Part 1 - Data Preprocessing


There are some concerns with the use of 'OneHotEncoder': 
1. https://axk51013.medium.com/不要再做one-hot-encoding-b5126d3f8a63

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [119]:
# Delete the 'RowNumber' , 'CustomerId' ,'Surname', 'Exited' 
X = dataset.iloc[:, 3:-1].values
Y = dataset.iloc[:,-1].values

In [120]:
X, Y

(array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
        [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
        [502, 'France', 'Female', ..., 1, 0, 113931.57],
        ...,
        [709, 'France', 'Female', ..., 0, 1, 42085.58],
        [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
        [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object),
 array([1, 0, 1, ..., 1, 1, 0]))

In [121]:
dataset.isna().sum() # No missing data! 

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

## Encoding the categorial data

In [122]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [123]:
X[:, 2] = le.fit_transform(X[:, 2])

In [124]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [125]:
X.shape

(10000, 10)

In [126]:
list(le.classes_)

['Female', 'Male']

In [127]:
# One Hot Enconding is not a good option since it creates lots of zeros and increases the dimension of the dataset 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [128]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[1] )] , remainder='passthrough' )
X = np.array(ct.fit_transform(X))

In [129]:
X, X.shape

(array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
        [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
        [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
        ...,
        [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
        [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
        [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object),
 (10000, 12))

In [130]:
print(X[0,:]) # move the 'country' to leftmost 

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]


## Split the dataset into Training and Test Set 

In [131]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=0)

## Feature Scaling

In [132]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [133]:
X_train = scaler.fit_transform(X_train)

In [134]:
X_test = scaler.fit_transform(X_test)

# Model Building 


In [93]:
ann = tf.keras.models.Sequential()

Metal device set to: Apple M1 Max


In [96]:
# Add Input layer
ann.add(tf.keras.layers.Dense(units = 6, activation='relu' ))

In [97]:
# Add one Hidden layer
ann.add(tf.keras.layers.Dense(units = 6, activation='relu' ))

In [98]:
# Add output layer
ann.add(tf.keras.layers.Dense(units = 1, activation='sigmoid' )) # output either 0/1 --> One Neuron 

# Part 3 - Train the ANN 


In [99]:
# Compile the ANN 
ann.compile( optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

In [101]:
# Train the ANN with training dataset 
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100


2023-04-27 22:30:06.081569: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


250/250 [==============================] - 3s 5ms/step - loss: 0.5674 - accuracy: 0.7186
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4640 - accuracy: 0.7961
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4497 - accuracy: 0.7969
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4423 - accuracy: 0.7975
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4370 - accuracy: 0.7989
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4325 - accuracy: 0.8018
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4263 - accuracy: 0.8035
Epoch 8/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4173 - accuracy: 0.8073
Epoch 9/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4049 - accuracy: 0.8135
Epoch 10/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3910 - accuracy: 0.8209
E

# Part 4 - Prediction

## testcase: 

Country = France 

Credit Score = 600 

Gender = Male 

Age = 40 Years Old 

Tenure: 3 years 

Balance: $60000

Number of Products: 2

Have Credit Card? Yes

Active Member? Yes

Expected Salary: 50000


In [135]:
test_case = [[1.0, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1,  50000]]

In [136]:
# Standardize the test_case
test_case_scaled = scaler.fit_transform(test_case)


In [137]:
ann.predict(test_case_scaled) # This customer is not likely to leave the bank

1/1 [==============================] - 0s 15ms/step


array([[0.09047633]], dtype=float32)

## Predicting the test result

In [138]:
y_pred = ann.predict(X_test)

63/63 [==============================] - 0s 2ms/step


In [141]:
y_pred = (y_pred > 0.5 )
np.concatenate((y_pred.reshape(len(y_pred),1) , y_test.reshape(len(y_test),1)  ),1)

array([[0, 0],
       [0, 1],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

## Confusion Matrix 

In [142]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[1507,   88],
       [ 192,  213]])

In [143]:
accuracy_score(y_test, y_pred)

0.86